<a href="https://colab.research.google.com/github/2303a51295madhuri/HPC-LAB/blob/main/2303A51295.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BARRIER**


barrier makes all threads wait until every thread reaches the same point.

No thread can go ahead until all arrive

In [66]:
%%writefile barrier.c
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();

        printf("Thread %d before barrier\n", id);

        #pragma omp barrier

        printf("Thread %d after barrier\n", id);
    }
    return 0;
}


Overwriting barrier.c


In [67]:
!gcc barrier.c -fopenmp -o barrier


In [68]:
!./barrier


Thread 1 before barrier
Thread 0 before barrier
Thread 1 after barrier
Thread 0 after barrier


**CRITICAL**

Allows only ONE thread at a time to execute a block.

Used to protect shared data from race condition.

In [69]:
%%writefile critical.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;

    #pragma omp parallel
    {
        #pragma omp critical
        {
            total++;
            printf("Thread %d -> total = %d\n",
                   omp_get_thread_num(), total);
        }
    }

    printf("Final total = %d\n", total);
    return 0;
}


Overwriting critical.c


In [70]:
!gcc critical.c -fopenmp -o critical


In [71]:
!./critical


Thread 0 -> total = 1
Thread 1 -> total = 2
Final total = 2


**ATOMIC**

Similar to critical but only for single simple statement

Faster than critical

In [72]:
%%writefile atomic.c
#include <stdio.h>
#include <omp.h>

int main() {
    int sum = 0;

    #pragma omp parallel
    {
        #pragma omp atomic
        sum += 1;
    }

    printf("Sum = %d\n", sum);
    return 0;
}


Overwriting atomic.c


In [73]:
!gcc atomic.c -fopenmp -o atomic


In [74]:
!./atomic


Sum = 2


**REDUCTION**

Used to combine results from all threads automatically.

Very useful in loops (sum, product, max, etc.)

In [75]:
%%writefile reduction.c
#include <stdio.h>
#include <omp.h>

int main() {
    int sum = 0;

    #pragma omp parallel for reduction(+:sum)
    for(int i = 1; i <= 10; i++) {
        sum += i;
    }

    printf("Reduction Sum = %d\n", sum);
    return 0;
}


Overwriting reduction.c


In [76]:
!gcc reduction.c -fopenmp -o reduction


In [77]:
!./reduction


Reduction Sum = 55


**ORDERED**

Used inside parallel loops

Ensures some part executes in sequential order

In [78]:
%%writefile ordered.c
#include <stdio.h>
#include <omp.h>

int main() {

    #pragma omp parallel for ordered
    for(int i = 0; i < 5; i++) {

        #pragma omp ordered
        {
            printf("Ordered Output: %d by thread %d\n",
                   i, omp_get_thread_num());
        }
    }

    return 0;
}


Overwriting ordered.c


In [79]:
!gcc ordered.c -fopenmp -o ordered


In [80]:
!./ordered


Ordered Output: 0 by thread 0
Ordered Output: 1 by thread 0
Ordered Output: 2 by thread 0
Ordered Output: 3 by thread 1
Ordered Output: 4 by thread 1


**MASTER**

Block executed only by master thread (thread 0)

Other threads skip it

In [81]:
%%writefile master.c
#include <stdio.h>
#include <omp.h>

int main() {

    #pragma omp parallel
    {
        #pragma omp master
        {
            printf("Executed only by master thread\n");
        }

        printf("Executed by all threads\n");
    }

    return 0;
}


Overwriting master.c


In [82]:
!gcc master.c -fopenmp -o master


In [83]:
!./master


Executed only by master thread
Executed by all threads
Executed by all threads


**SINGLE**

Only one arbitrary thread executes the block

Not necessarily master thread

In [84]:
%%writefile single.c
#include <stdio.h>
#include <omp.h>

int main() {

    #pragma omp parallel
    {
        #pragma omp single
        {
            printf("Executed by only ONE thread\n");
        }

        printf("Executed by all threads\n");
    }

    return 0;
}


Overwriting single.c


In [85]:
!gcc single.c -fopenmp -o single


In [86]:
!./single


Executed by only ONE thread
Executed by all threads
Executed by all threads


**FLUSH**

Ensures that a variable updated by one thread becomes visible to other threads.

Used for memory synchronization

In [87]:
%%writefile flush.c
#include <stdio.h>
#include <omp.h>

int main() {

    int flag = 0;

    #pragma omp parallel num_threads(2)
    {
        int id = omp_get_thread_num();

        if(id == 0) {
            // Thread 0 updates flag
            flag = 1;

            // make change visible to other thread
            #pragma omp flush(flag)

            printf("Thread 0 changed flag to 1\n");
        }
        else {
            // Thread 1 waits until flag becomes 1
            while(flag == 0) {
                #pragma omp flush(flag)
            }

            printf("Thread 1 detected flag change\n");
        }
    }

    return 0;
}


Overwriting flush.c


In [88]:
!gcc flush.c -fopenmp -o flush


In [89]:
!./flush


Thread 1 detected flag change
Thread 0 changed flag to 1


openmp(lock)


In [90]:
%%writefile lock_example.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;
    omp_lock_t lock;

    // Initialize the lock
    omp_init_lock(&lock);

    #pragma omp parallel num_threads(4)
    {
        for(int i = 0; i < 5; i++) {
            // Acquire lock before updating total
            omp_set_lock(&lock);
            total++;
            printf("Thread %d updated total = %d\n", omp_get_thread_num(), total);
            omp_unset_lock(&lock);  // Release lock
        }
    }

    // Destroy the lock
    omp_destroy_lock(&lock);

    printf("Final total = %d\n", total);
    return 0;
}


Writing lock_example.c


In [91]:
!gcc lock_example.c -fopenmp -o lock_example


In [92]:
!./lock_example


Thread 3 updated total = 1
Thread 3 updated total = 2
Thread 3 updated total = 3
Thread 3 updated total = 4
Thread 3 updated total = 5
Thread 1 updated total = 6
Thread 1 updated total = 7
Thread 1 updated total = 8
Thread 1 updated total = 9
Thread 1 updated total = 10
Thread 2 updated total = 11
Thread 2 updated total = 12
Thread 2 updated total = 13
Thread 2 updated total = 14
Thread 2 updated total = 15
Thread 0 updated total = 16
Thread 0 updated total = 17
Thread 0 updated total = 18
Thread 0 updated total = 19
Thread 0 updated total = 20
Final total = 20


openmp(nolock)

In [93]:
%%writefile nolock_example.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;

    #pragma omp parallel num_threads(4)
    {
        for(int i = 0; i < 5; i++) {
            // No lock here
            total++;
            printf("Thread %d updated total = %d\n", omp_get_thread_num(), total);
        }
    }

    printf("Final total = %d\n", total);
    return 0;
}


Writing nolock_example.c


In [94]:
!gcc nolock_example.c -fopenmp -o nolock_example


In [95]:
!./nolock_example


Thread 2 updated total = 1
Thread 2 updated total = 3
Thread 2 updated total = 4
Thread 3 updated total = 2
Thread 3 updated total = 6
Thread 3 updated total = 7
Thread 3 updated total = 8
Thread 3 updated total = 9
Thread 1 updated total = 10
Thread 1 updated total = 11
Thread 1 updated total = 12
Thread 1 updated total = 13
Thread 1 updated total = 14
Thread 2 updated total = 5
Thread 2 updated total = 15
Thread 0 updated total = 16
Thread 0 updated total = 17
Thread 0 updated total = 18
Thread 0 updated total = 19
Thread 0 updated total = 20
Final total = 20
